In [1]:
# !git clone -b predict https://github.com/K4TEL/geo-twitter.git

In [1]:
# ! pip install -r ./geo-twitter/requirements.txt

In [1]:
import os
os.environ['PYSPARK_PYTHON'] = "/opt/anaconda3/bin/python"
os.environ['PYSPARK_DRIVER_PYTHON'] = "/opt/anaconda3/bin/python"

In [124]:
# ! /opt/anaconda3/bin/python -c "import torch"
#export PYSPARK_PYTHON=/opt/anaconda3/bin/python
updated_df.select('user.continent').show(10)

+-------------+
|    continent|
+-------------+
|North America|
|       Africa|
|North America|
|North America|
|         Asia|
|North America|
|North America|
|         Asia|
|       Europe|
|       Africa|
+-------------+
only showing top 10 rows



In [126]:
updated_df.select('projects_following').show(10)

+--------------------+
|  projects_following|
+--------------------+
| [{PFPs, code BLUE}]|
|[{PFPs, Chill Chi...|
|[{Dexes, Potluck ...|
|[{Art, Generascop...|
|[{Liquid Staking,...|
|[{Dexes, Juggler ...|
|[{Art, Platform 9...|
|[{Art, Future Rea...|
|[{Photography, ra...|
|[{Gaming, Spinbla...|
+--------------------+
only showing top 10 rows



In [127]:
updated_df.select('tweets.sentiment').show(10)

+--------------------+
|           sentiment|
+--------------------+
|[{neutral, 0.7515...|
|[{neutral, 0.5239...|
|[{neutral, 0.4977...|
|[{neutral, 0.4387...|
|[{neutral, 0.5958...|
|[{positive, 0.725...|
|[{neutral, 0.6935...|
|[{neutral, 0.3675...|
|[{positive, 0.852...|
|[{neutral, 0.8235...|
+--------------------+
only showing top 10 rows



In [57]:
spark.stop()

# Connect SPARK

In [2]:
##Configuration
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
#config the connector jar file
spark = (SparkSession.builder.appName("SimpleSparkJob").master("spark://34.142.194.212:7077")
         .config("spark.jars", "/opt/spark/jars/gcs-connector-latest-hadoop2.jar")
         .config("spark.executor.memory", "2G")  #excutor excute only 2G
        .config("spark.driver.memory","4G") 
        .config("spark.executor.cores","1") #Cluster use only 3 cores to excute as it has 3 server
        .config("spark.python.worker.memory","1G") # each worker use 1G to excute
        .config("spark.driver.maxResultSize","3G") #Maximum size of result is 3G
        .config("spark.kryoserializer.buffer.max","1024M")
        .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") 
        .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") 
        #  .config("spark.shuffle.service.enabled", "false")
        # .config("spark.dynamicAllocation.enabled", "false")
        # .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", "/opt/bucket_connector/lucky-wall-393304-2a6a3df38253.json") 
        # .config("spark.jars.packages", "com.google.cloud.bigdataoss:gcs-connector:hadoop3-2.2.0")

         .getOrCreate())
#config the credential to identify the google cloud hadoop file 
# spark.conf.set("google.cloud.auth.service.account.json.keyfile","/opt/bucket_connector/lucky-wall-393304-2a6a3df38253.json")
# spark._jsc.hadoopConfiguration().set('fs.gs.impl', 'com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem')
# spark._jsc.hadoopConfiguration().set('fs.gs.auth.service.account.enable', 'true')

## Connect to the file in Google Bucket with Spark

path= f"gs://it4043e-it5384/it5384/IT5384_Group2_Problem1/Data1.json"
df =spark.read.json(path, multiLine=True)
df.show()

#spark.stop() # Ending spark job

24/01/01 23:58:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/01 23:58:22 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+--------------------+--------------------+--------------------+
|  projects_following|              tweets|                user|
+--------------------+--------------------+--------------------+
| [{PFPs, code BLUE}]|[{0, [], 0, [], N...|{human, [11, 3, 0...|
|[{Algo-Stables, S...|[{0, [], 0, [{sns...|{human, [17, 11, ...|
|[{PFPs, Chill Chi...|[{24, [tezos], 18...|{human, [7, 5, 23...|
|[{Dexes, Potluck ...|[{24, [], 53, [{s...|{human, [21, 16, ...|
|[{Art, Generascop...|[{5, [], 32, [], ...|{human, [15, 19, ...|
|[{Liquid Staking,...|[{0, [], 0, [{sns...|{human, [18, 5, 1...|
|[{Dexes, Juggler ...|[{0, [], 6, [{sns...|{human, [16, 17, ...|
|[{Launchpad, Seed...|[{22, [UE5, Cryow...|{human, [16, 15, ...|
|[{Art, Platform 9...|[{47, [], 299, [{...|{human, [23, 9, 3...|
|[{PFPs, Sugar Hea...|[{0, [], 0, [{sns...|{human, [12, 12, ...|
|[{Art, Future Rea...|[{86, [], 30274, ...|{human, [14, 13, ...|
|[{PFPs, AI Catgirl}]|[{2, [minecraft, ...|{human, [15, 22, ...|
| [{Yield, Lavafall}]|[{0

In [3]:
path= f"gs://it4043e-it5384/it5384/IT5384_Group2_Problem1/FinalBigData.json"
dfData =spark.read.json(path, multiLine=True)

In [4]:
spark

# Location Predicting 

In [5]:
import sys
print(sys.executable)

/opt/anaconda3/bin/python


In [6]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

sc.addPyFile("text_result.py")

In [7]:
from pyspark import SparkConf
conf = SparkConf().set("spark.executorEnv.PYSPARK_PYTHON", "/opt/anaconda3/bin/python")

In [8]:
# import sys
# sys.path.append('./geo-twitter')
# import text_result
from text_result import load_model, text_prediction

In [9]:
hub_model = 'k4tel/geo-bert-multilingual'
base_model = "bert-base-multilingual-cased"
use_pipeline = True

In [10]:
model_wrapper = load_model(base_model, hub_model, use_pipeline)

MODEL	Initializing BERT Regression model for 5 outcome(s)
MODEL	Text features:	NON-GEO + GEO-ONLY
MODEL	Coordinates:	10
MODEL	Weights:	5
MODEL	Covariances:	5	matrix type:	spher
MODEL	Original model to load:	bert-base-multilingual-cased
MODEL	Key feature 	NON-GEO outputs:	20
MODEL	Minor feature	GEO-ONLY outputs:	3
LOAD	Loading HF model from k4tel/geo-bert-multilingual


In [11]:
text = "Chi la dura la vince url"
filter = False

In [12]:
import numpy as np
result = text_prediction(model_wrapper, text, use_pipeline, filter)
ind = np.argwhere(np.round(result.weights[0, :] * 100, 2) > 0)
significant = result.means[0, ind].reshape(-1, 2)
weights = result.weights[0, ind].flatten()

sig_weights = weights[weights > 0]

print(f"RESULT\t{len(sig_weights)} significant prediction outcome(s):")
for i in range(len(sig_weights)):
    point = f"lon: {'  lat: '.join(map(str, significant[i]))}"
    weight = str(np.round(sig_weights[i] * 100, 2))
    print(f"\tOut {i + 1}\t{weight}%\t-\t{point}")

RESULT	Post-processing raw model outputs: tensor([[  4.4452,  25.6701,   9.3040,  40.9892,  13.7981,  41.2131,  20.4728,
          -3.0671,  20.0061,  21.3602, -19.0938,  -1.4898,   4.5327, -18.8960,
         -18.1328,   3.3912, 158.3822,   1.4718,  -5.1619,  46.7757]])
RESULT	Sorting all outputs for 5 outcomes by probabilistic weights
RESULT	2 significant prediction outcome(s):
	Out 1	99.76%	-	lon: 13.798074722290039  lat: 41.213130950927734
	Out 2	0.24%	-	lon: 9.304001808166504  lat: 40.989173889160156


In [13]:
# spark.stop()

In [14]:
country_bboxes = {
    'AF': ('Afghanistan', (60.5284298033, 29.318572496, 75.1580277851, 38.4862816432)),
    'AO': ('Angola', (11.6400960629, -17.9306364885, 24.0799052263, -4.43802336998)),
    'AL': ('Albania', (19.3044861183, 39.624997667, 21.0200403175, 42.6882473822)),
    'AE': ('United Arab Emirates', (51.5795186705, 22.4969475367, 56.3968473651, 26.055464179)),
    'AR': ('Argentina', (-73.4154357571, -55.25, -53.628348965, -21.8323104794)),
    'AM': ('Armenia', (43.5827458026, 38.7412014837, 46.5057198423, 41.2481285671)),
    'AQ': ('Antarctica', (-180.0, -90.0, 180.0, -63.2706604895)),
    'TF': ('Fr. S. and Antarctic Lands', (68.72, -49.775, 70.56, -48.625)),
    'AU': ('Australia', (113.338953078, -43.6345972634, 153.569469029, -10.6681857235)),
    'AT': ('Austria', (9.47996951665, 46.4318173285, 16.9796667823, 49.0390742051)),
    'AZ': ('Azerbaijan', (44.7939896991, 38.2703775091, 50.3928210793, 41.8606751572)),
    'BI': ('Burundi', (29.0249263852, -4.49998341229, 30.752262811, -2.34848683025)),
    'BE': ('Belgium', (2.51357303225, 49.5294835476, 6.15665815596, 51.4750237087)),
    'BJ': ('Benin', (0.772335646171, 6.14215770103, 3.79711225751, 12.2356358912)),
    'BF': ('Burkina Faso', (-5.47056494793, 9.61083486576, 2.17710778159, 15.1161577418)),
    'BD': ('Bangladesh', (88.0844222351, 20.670883287, 92.6727209818, 26.4465255803)),
    'BG': ('Bulgaria', (22.3805257504, 41.2344859889, 28.5580814959, 44.2349230007)),
    'BS': ('Bahamas', (-78.98, 23.71, -77.0, 27.04)),
    'BA': ('Bosnia and Herz.', (15.7500260759, 42.65, 19.59976, 45.2337767604)),
    'BY': ('Belarus', (23.1994938494, 51.3195034857, 32.6936430193, 56.1691299506)),
    'BZ': ('Belize', (-89.2291216703, 15.8869375676, -88.1068129138, 18.4999822047)),
    'BO': ('Bolivia', (-69.5904237535, -22.8729187965, -57.4983711412, -9.76198780685)),
    'BR': ('Brazil', (-73.9872354804, -33.7683777809, -34.7299934555, 5.24448639569)),
    'BN': ('Brunei', (114.204016555, 4.007636827, 115.450710484, 5.44772980389)),
    'BT': ('Bhutan', (88.8142484883, 26.7194029811, 92.1037117859, 28.2964385035)),
    'BW': ('Botswana', (19.8954577979, -26.8285429827, 29.4321883481, -17.6618156877)),
    'CF': ('Central African Rep.', (14.4594071794, 2.2676396753, 27.3742261085, 11.1423951278)),
    'CA': ('Canada', (-140.99778, 41.6751050889, -52.6480987209, 83.23324)),
    'CH': ('Switzerland', (6.02260949059, 45.7769477403, 10.4427014502, 47.8308275417)),
    'CL': ('Chile', (-75.6443953112, -55.61183, -66.95992, -17.5800118954)),
    'CN': ('China', (73.6753792663, 18.197700914, 135.026311477, 53.4588044297)),
    'CI': ('Ivory Coast', (-8.60288021487, 4.33828847902, -2.56218950033, 10.5240607772)),
    'CM': ('Cameroon', (8.48881554529, 1.72767263428, 16.0128524106, 12.8593962671)),
    'CD': ('Congo (Kinshasa)', (12.1823368669, -13.2572266578, 31.1741492042, 5.25608775474)),
    'CG': ('Congo (Brazzaville)', (11.0937728207, -5.03798674888, 18.4530652198, 3.72819651938)),
    'CO': ('Colombia', (-78.9909352282, -4.29818694419, -66.8763258531, 12.4373031682)),
    'CR': ('Costa Rica', (-85.94172543, 8.22502798099, -82.5461962552, 11.2171192489)),
    'CU': ('Cuba', (-84.9749110583, 19.8554808619, -74.1780248685, 23.1886107447)),
    'CY': ('Cyprus', (32.2566671079, 34.5718694118, 34.0048808123, 35.1731247015)),
    'CZ': ('Czech Rep.', (12.2401111182, 48.5553052842, 18.8531441586, 51.1172677679)),
    'DE': ('Germany', (5.98865807458, 47.3024876979, 15.0169958839, 54.983104153)),
    'DJ': ('Djibouti', (41.66176, 10.9268785669, 43.3178524107, 12.6996385767)),
    'DK': ('Denmark', (8.08997684086, 54.8000145534, 12.6900061378, 57.730016588)),
    'DO': ('Dominican Rep.', (-71.9451120673, 17.598564358, -68.3179432848, 19.8849105901)),
    'DZ': ('Algeria', (-8.68439978681, 19.0573642034, 11.9995056495, 37.1183806422)),
    'EC': ('Ecuador', (-80.9677654691, -4.95912851321, -75.2337227037, 1.3809237736)),
    'EG': ('Egypt', (24.70007, 22.0, 36.86623, 31.58568)),
    'ER': ('Eritrea', (36.3231889178, 12.4554157577, 43.0812260272, 17.9983074)),
    'ES': ('Spain', (-9.39288367353, 35.946850084, 3.03948408368, 43.7483377142)),
    'EE': ('Estonia', (23.3397953631, 57.4745283067, 28.1316992531, 59.6110903998)),
    'ET': ('Ethiopia', (32.95418, 3.42206, 47.78942, 14.95943)),
    'FI': ('Finland', (20.6455928891, 59.846373196, 31.5160921567, 70.1641930203)),
    'FJ': ('Fiji', (-180.0, -18.28799, 180.0, -16.0208822567)),
    'FK': ('Falkland Is.', (-61.2, -52.3, -57.75, -51.1)),
    'FR': ('France', (-54.5247541978, 2.05338918702, 9.56001631027, 51.1485061713)),
    'GA': ('Gabon', (8.79799563969, -3.97882659263, 14.4254557634, 2.32675751384)),
    'GB': ('United Kingdom', (-7.57216793459, 49.959999905, 1.68153079591, 58.6350001085)),
    'GE': ('Georgia', (39.9550085793, 41.0644446885, 46.6379081561, 43.553104153)),
    'GH': ('Ghana', (-3.24437008301, 4.71046214438, 1.0601216976, 11.0983409693)),
    'GN': ('Guinea', (-15.1303112452, 7.3090373804, -7.83210038902, 12.5861829696)),
    'GM': ('Gambia', (-16.8415246241, 13.1302841252, -13.8449633448, 13.8764918075)),
    'GW': ('Guinea Bissau', (-16.6774519516, 11.0404116887, -13.7004760401, 12.6281700708)),
    'GQ': ('Eq. Guinea', (9.3056132341, 1.01011953369, 11.285078973, 2.28386607504)),
    'GR': ('Greece', (20.1500159034, 34.9199876979, 26.6041955909, 41.8269046087)),
    'GL': ('Greenland', (-73.297, 60.03676, -12.20855, 83.64513)),
    'GT': ('Guatemala', (-92.2292486234, 13.7353376327, -88.2250227526, 17.8193260767)),
    'GY': ('Guyana', (-61.4103029039, 1.26808828369, -56.5393857489, 8.36703481692)),
    'HN': ('Honduras', (-89.3533259753, 12.9846857772, -83.147219001, 16.0054057886)),
    'HR': ('Croatia', (13.6569755388, 42.47999136, 19.3904757016, 46.5037509222)),
    'HT': ('Haiti', (-74.4580336168, 18.0309927434, -71.6248732164, 19.9156839055)),
    'HU': ('Hungary', (16.2022982113, 45.7594811061, 22.710531447, 48.6238540716)),
    'ID': ('Indonesia', (95.2930261576, -10.3599874813, 141.03385176, 5.47982086834)),
    'IN': ('India', (68.1766451354, 7.96553477623, 97.4025614766, 35.4940095078)),
    'IE': ('Ireland', (-9.97708574059, 51.6693012559, -6.03298539878, 55.1316222195)),
    'IR': ('Iran', (44.1092252948, 25.0782370061, 63.3166317076, 39.7130026312)),
    'IQ': ('Iraq', (38.7923405291, 29.0990251735, 48.5679712258, 37.3852635768)),
    'IS': ('Iceland', (-24.3261840479, 63.4963829617, -13.609732225, 66.5267923041)),
    'IL': ('Israel', (34.2654333839, 29.5013261988, 35.8363969256, 33.2774264593)),
    'IT': ('Italy', (6.7499552751, 36.619987291, 18.4802470232, 47.1153931748)),
    'JM': ('Jamaica', (-78.3377192858, 17.7011162379, -76.1996585761, 18.5242184514)),
    'JO': ('Jordan', (34.9226025734, 29.1974946152, 39.1954683774, 33.3786864284)),
    'JP': ('Japan', (129.408463169, 31.0295791692, 145.543137242, 45.5514834662)),
    'KZ': ('Kazakhstan', (46.4664457538, 40.6623245306, 87.3599703308, 55.3852501491)),
    'KE': ('Kenya', (33.8935689697, -4.67677, 41.8550830926, 5.506)),
    'KG': ('Kyrgyzstan', (69.464886916, 39.2794632025, 80.2599902689, 43.2983393418)),
    'KH': ('Cambodia', (102.3480994, 10.4865436874, 107.614547968, 14.5705838078)),
    'KR': ('S. Korea', (126.117397903, 34.3900458847, 129.468304478, 38.6122429469)),
    'KW': ('Kuwait', (46.5687134133, 28.5260627304, 48.4160941913, 30.0590699326)),
    'LA': ('Laos', (100.115987583, 13.88109101, 107.564525181, 22.4647531194)),
    'LB': ('Lebanon', (35.1260526873, 33.0890400254, 36.6117501157, 34.6449140488)),
    'LR': ('Liberia', (-11.4387794662, 4.35575511313, -7.53971513511, 8.54105520267)),
    'LY': ('Libya', (9.31941084152, 19.58047, 25.16482, 33.1369957545)),
    'LK': ('Sri Lanka', (79.6951668639, 5.96836985923, 81.7879590189, 9.82407766361)),
    'LS': ('Lesotho', (26.9992619158, -30.6451058896, 29.3251664568, -28.6475017229)),
    'LT': ('Lithuania', (21.0558004086, 53.9057022162, 26.5882792498, 56.3725283881)),
    'LU': ('Luxembourg', (5.67405195478, 49.4426671413, 6.24275109216, 50.1280516628)),
    'LV': ('Latvia', (21.0558004086, 55.61510692, 28.1767094256, 57.9701569688)),
    'MA': ('Morocco', (-17.0204284327, 21.4207341578, -1.12455115397, 35.7599881048)),
    'MD': ('Moldova', (26.6193367856, 45.4882831895, 30.0246586443, 48.4671194525)),
    'MG': ('Madagascar', (43.2541870461, -25.6014344215, 50.4765368996, -12.0405567359)),
    'MX': ('Mexico', (-117.12776, 14.5388286402, -86.811982388, 32.72083)),
    'MK': ('Macedonia', (20.46315, 40.8427269557, 22.9523771502, 42.3202595078)),
    'ML': ('Mali', (-12.1707502914, 10.0963607854, 4.27020999514, 24.9745740829)),
    'MM': ('Myanmar', (92.3032344909, 9.93295990645, 101.180005324, 28.335945136)),
    'ME': ('Montenegro', (18.45, 41.87755, 20.3398, 43.52384)),
    'MN': ('Mongolia', (87.7512642761, 41.5974095729, 119.772823928, 52.0473660345)),
    'MZ': ('Mozambique', (30.1794812355, -26.7421916643, 40.7754752948, -10.3170960425)),
    'MR': ('Mauritania', (-17.0634232243, 14.6168342147, -4.92333736817, 27.3957441269)),
    'MW': ('Malawi', (32.6881653175, -16.8012997372, 35.7719047381, -9.23059905359)),
    'MY': ('Malaysia', (100.085756871, 0.773131415201, 119.181903925, 6.92805288332)),
    'NA': ('Namibia', (11.7341988461, -29.045461928, 25.0844433937, -16.9413428687)),
    'NC': ('New Caledonia', (164.029605748, -22.3999760881, 167.120011428, -20.1056458473)),
    'NE': ('Niger', (0.295646396495, 11.6601671412, 15.9032466977, 23.4716684026)),
    'NG': ('Nigeria', (2.69170169436, 4.24059418377, 14.5771777686, 13.8659239771)),
    'NI': ('Nicaragua', (-87.6684934151, 10.7268390975, -83.147219001, 15.0162671981)),
    'NL': ('Netherlands', (3.31497114423, 50.803721015, 7.09205325687, 53.5104033474)),
    'NO': ('Norway', (4.99207807783, 58.0788841824, 31.29341841, 80.6571442736)),
    'NP': ('Nepal', (80.0884245137, 26.3978980576, 88.1748043151, 30.4227169866)),
    'NZ': ('New Zealand', (166.509144322, -46.641235447, 178.517093541, -34.4506617165)),
    'OM': ('Oman', (52.0000098, 16.6510511337, 59.8080603372, 26.3959343531)),
    'PK': ('Pakistan', (60.8742484882, 23.6919650335, 77.8374507995, 37.1330309108)),
    'PA': ('Panama', (-82.9657830472, 7.2205414901, -77.2425664944, 9.61161001224)),
    'PE': ('Peru', (-81.4109425524, -18.3479753557, -68.6650797187, -0.0572054988649)),
    'PH': ('Philippines', (117.17427453, 5.58100332277, 126.537423944, 18.5052273625)),
    'PG': ('Papua New Guinea', (141.000210403, -10.6524760881, 156.019965448, -2.50000212973)),
    'PL': ('Poland', (14.0745211117, 49.0273953314, 24.0299857927, 54.8515359564)),
    'PR': ('Puerto Rico', (-67.2424275377, 17.946553453, -65.5910037909, 18.5206011011)),
    'KP': ('N. Korea', (124.265624628, 37.669070543, 130.780007359, 42.9853868678)),
    'PT': ('Portugal', (-9.52657060387, 36.838268541, -6.3890876937, 42.280468655)),
    'PY': ('Paraguay', (-62.6850571357, -27.5484990374, -54.2929595608, -19.3427466773)),
    'QA': ('Qatar', (50.7439107603, 24.5563308782, 51.6067004738, 26.1145820175)),
    'RO': ('Romania', (20.2201924985, 43.6884447292, 29.62654341, 48.2208812526)),
    'RU': ('Russia', (-180.0, 41.151416124, 180.0, 81.2504)),
    'RW': ('Rwanda', (29.0249263852, -2.91785776125, 30.8161348813, -1.13465911215)),
    'SA': ('Saudi Arabia', (34.6323360532, 16.3478913436, 55.6666593769, 32.161008816)),
    'SD': ('Sudan', (21.93681, 8.61972971293, 38.4100899595, 22.0)),
    'SS': ('S. Sudan', (23.8869795809, 3.50917, 35.2980071182, 12.2480077571)),
    'SN': ('Senegal', (-17.6250426905, 12.332089952, -11.4678991358, 16.5982636581)),
    'SB': ('Solomon Is.', (156.491357864, -10.8263672828, 162.398645868, -6.59933847415)),
    'SL': ('Sierra Leone', (-13.2465502588, 6.78591685631, -10.2300935531, 10.0469839543)),
    'SV': ('El Salvador', (-90.0955545723, 13.1490168319, -87.7235029772, 14.4241327987)),
    'SO': ('Somalia', (40.98105, -1.68325, 51.13387, 12.02464)),
    'RS': ('Serbia', (18.82982, 42.2452243971, 22.9860185076, 46.1717298447)),
    'SR': ('Suriname', (-58.0446943834, 1.81766714112, -53.9580446031, 6.0252914494)),
    'SK': ('Slovakia', (16.8799829444, 47.7584288601, 22.5581376482, 49.5715740017)),
    'SI': ('Slovenia', (13.6981099789, 45.4523163926, 16.5648083839, 46.8523859727)),
    'SE': ('Sweden', (11.0273686052, 55.3617373725, 23.9033785336, 69.1062472602)),
    'SZ': ('Swaziland', (30.6766085141, -27.2858794085, 32.0716654803, -25.660190525)),
    'SY': ('Syria', (35.7007979673, 32.312937527, 42.3495910988, 37.2298725449)),
    'TD': ('Chad', (13.5403935076, 7.42192454674, 23.88689, 23.40972)),
    'TG': ('Togo', (-0.0497847151599, 5.92883738853, 1.86524051271, 11.0186817489)),
    'TH': ('Thailand', (97.3758964376, 5.69138418215, 105.589038527, 20.4178496363)),
    'TJ': ('Tajikistan', (67.4422196796, 36.7381712916, 74.9800024759, 40.9602133245)),
    'TM': ('Turkmenistan', (52.5024597512, 35.2706639674, 66.5461503437, 42.7515510117)),
    'TL': ('East Timor', (124.968682489, -9.39317310958, 127.335928176, -8.27334482181)),
    'TT': ('Trinidad and Tobago', (-61.95, 10.0, -60.895, 10.89)),
    'TN': ('Tunisia', (7.52448164229, 30.3075560572, 11.4887874691, 37.3499944118)),
    'TR': ('Turkey', (26.0433512713, 35.8215347357, 44.7939896991, 42.1414848903)),
    'TW': ('Taiwan', (120.106188593, 21.9705713974, 121.951243931, 25.2954588893)),
    'TZ': ('Tanzania', (29.3399975929, -11.7209380022, 40.31659, -0.95)),
    'UG': ('Uganda', (29.5794661801, -1.44332244223, 35.03599, 4.24988494736)),
    'UA': ('Ukraine', (22.0856083513, 44.3614785833, 40.0807890155, 52.3350745713)),
    'UY': ('Uruguay', (-58.4270741441, -34.9526465797, -53.209588996, -30.1096863746)),
    'US': ('United States', (-171.791110603, 18.91619, -66.96466, 71.3577635769)),
    'UZ': ('Uzbekistan', (55.9289172707, 37.1449940049, 73.055417108, 45.5868043076)),
    'VE': ('Venezuela', (-73.3049515449, 0.724452215982, -59.7582848782, 12.1623070337)),
    'VN': ('Vietnam', (102.170435826, 8.59975962975, 109.33526981, 23.3520633001)),
    'VU': ('Vanuatu', (166.629136998, -16.5978496233, 167.844876744, -14.6264970842)),
    'PS': ('West Bank', (34.9274084816, 31.3534353704, 35.5456653175, 32.5325106878)),
    'YE': ('Yemen', (42.6048726743, 12.5859504257, 53.1085726255, 19.0000033635)),
    'ZA': ('South Africa', (16.3449768409, -34.8191663551, 32.830120477, -22.0913127581)),
    'ZM': ('Zambia', (21.887842645, -17.9612289364, 33.4856876971, -8.23825652429)),
    'ZW': ('Zimbabwe', (25.2642257016, -22.2716118303, 32.8498608742, -15.5077869605)),}

In [15]:
import geonamescache
gc = geonamescache.GeonamesCache()
countries = gc.get_countries()
continents = gc.get_continents()
# Broadcast the dictionaries
broadcast_countries = spark.sparkContext.broadcast(countries)
broadcast_continents = spark.sparkContext.broadcast(continents)
#continent_names = {code: continent['toponymName'] for code, continent in continents.items()}
continent_names = spark.sparkContext.broadcast(
    {code: continent['toponymName'] for code, continent in gc.get_continents().items()})

In [16]:
# Assuming continent_names is a broadcast variable
# country_to_continent will not be a broadcast variable but a regular dictionary
country_to_continent = {
    country['iso']: continent_names.value[country['continentcode']] 
    for country in countries.values()
}

# If you need to broadcast country_to_continent
country_to_continent = spark.sparkContext.broadcast(country_to_continent)

In [17]:
continent_names.value

{'AF': 'Africa',
 'AS': 'Asia',
 'EU': 'Europe',
 'NA': 'North America',
 'OC': 'Oceania',
 'SA': 'South America',
 'AN': 'Antarctica'}

In [18]:
from pyspark.sql.functions import udf

country_bboxes_broadcast = spark.sparkContext.broadcast(country_bboxes)

def get_country_from_coords(longitude, latitude):
    country_bboxes = country_bboxes_broadcast.value
    for country_code, (country_name, (west, south, east, north)) in country_bboxes.items():
        if west <= longitude <= east and south <= latitude <= north:
            return country_code
    return "Unknown"

get_country_from_coords_udf = udf(get_country_from_coords, StringType())

In [19]:
def get_continent_from_country(country_code):
    # Accessing the broadcasted dictionary using .value
    country_to_continent_dict = country_to_continent.value
    return country_to_continent_dict.get(country_code, "Unknown")


get_continent_from_country_udf = udf(get_continent_from_country, StringType())

## Stage 1: using decoding nominatim

In [20]:
from pyspark.sql.types import StringType
import requests

def get_location_tag_udf(location):
    try:
        url = "https://nominatim.openstreetmap.org/search"
        params = {
            "q": location,
            "format": "json"
        }
        response = requests.get(url, params=params)
        results = response.json()
        if results:
            loc = results[0]
            lat, lon = loc['lat'], loc['lon']

            # Reverse geocode to get country code
            rev_url = "https://nominatim.openstreetmap.org/reverse"
            rev_params = {
                "lat": lat,
                "lon": lon,
                "format": "json"
            }
            rev_response = requests.get(rev_url, params=rev_params)
            rev_results = rev_response.json()
            if rev_results and 'address' in rev_results and 'country_code' in rev_results['address']:
                countryCode = rev_results['address']['country_code'].upper()

                # Use broadcast variables
                countries_dict = broadcast_countries.value
                continents_dict = broadcast_continents.value

                for country in countries_dict:
                    if countries_dict[country]['iso'] == countryCode:
                        continentCode = countries_dict[country]['continentcode']
                        if continentCode in continents_dict:
                            return continents_dict[continentCode]['toponymName']
                        break
    except Exception as e:
        return "Unknown"

get_location_tag_spark_udf = udf(get_location_tag_udf, StringType())

In [21]:
df_with_continent = df.withColumn("continent", get_location_tag_spark_udf(df["user.location"]))

In [22]:
df_with_continent.show()

24/01/02 00:01:27 WARN TaskSetManager: Lost task 0.0 in stage 3.0 (TID 3) (10.148.0.22 executor 1): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 1100, in main
    raise PySparkRuntimeError(
pyspark.errors.exceptions.base.PySparkRuntimeError: [PYTHON_VERSION_MISMATCH] Python in worker has different version (3, 11) than that in driver 3.9, PySpark cannot run with different minor versions.
Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:572)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:94)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$1.read(PythonUDFRunner.scala:75)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:525)
	at 

+--------------------+--------------------+--------------------+-------------+
|  projects_following|              tweets|                user|    continent|
+--------------------+--------------------+--------------------+-------------+
| [{PFPs, code BLUE}]|[{0, [], 0, [], N...|{human, [11, 3, 0...|       Europe|
|[{Algo-Stables, S...|[{0, [], 0, [{sns...|{human, [17, 11, ...|         NULL|
|[{PFPs, Chill Chi...|[{24, [tezos], 18...|{human, [7, 5, 23...|       Africa|
|[{Dexes, Potluck ...|[{24, [], 53, [{s...|{human, [21, 16, ...|North America|
|[{Art, Generascop...|[{5, [], 32, [], ...|{human, [15, 19, ...|North America|
|[{Liquid Staking,...|[{0, [], 0, [{sns...|{human, [18, 5, 1...|         Asia|
|[{Dexes, Juggler ...|[{0, [], 6, [{sns...|{human, [16, 17, ...|North America|
|[{Launchpad, Seed...|[{22, [UE5, Cryow...|{human, [16, 15, ...|         NULL|
|[{Art, Platform 9...|[{47, [], 299, [{...|{human, [23, 9, 3...|North America|
|[{PFPs, Sugar Hea...|[{0, [], 0, [{sns...|{human, [

In [23]:
from pyspark.sql.functions import col, size
#Filter projects_following field not null
newDf = df_with_continent.filter((size(col('projects_following')) > 0))
newDf.show()

+--------------------+--------------------+--------------------+-------------+
|  projects_following|              tweets|                user|    continent|
+--------------------+--------------------+--------------------+-------------+
| [{PFPs, code BLUE}]|[{0, [], 0, [], N...|{human, [11, 3, 0...|       Europe|
|[{Algo-Stables, S...|[{0, [], 0, [{sns...|{human, [17, 11, ...|         NULL|
|[{PFPs, Chill Chi...|[{24, [tezos], 18...|{human, [7, 5, 23...|       Africa|
|[{Dexes, Potluck ...|[{24, [], 53, [{s...|{human, [21, 16, ...|North America|
|[{Art, Generascop...|[{5, [], 32, [], ...|{human, [15, 19, ...|North America|
|[{Liquid Staking,...|[{0, [], 0, [{sns...|{human, [18, 5, 1...|         Asia|
|[{Dexes, Juggler ...|[{0, [], 6, [{sns...|{human, [16, 17, ...|North America|
|[{Launchpad, Seed...|[{22, [UE5, Cryow...|{human, [16, 15, ...|         NULL|
|[{Art, Platform 9...|[{47, [], 299, [{...|{human, [23, 9, 3...|North America|
|[{PFPs, Sugar Hea...|[{0, [], 0, [{sns...|{human, [

## Stage 2: using geo-bert predicting

In [24]:
# from pyspark.sql.types import StringType
# from pyspark.sql.functions import col, when, udf
# import numpy as np

# def geo_bert_continent(tweets_array):
#     continent_votes = []

#     for tweet in tweets_array:
#         try: 
#             tweet_text = tweet['tweet_content'] 
#             if(len(tweet_text)>2500):
#                 tweet_text= tweet_text[:2500]
#             result = text_prediction(model_wrapper, tweet_text, True)
#             #print(result)
#             if result and result.weights.size > 0:
#                 max_weight_index = np.argmax(result.weights[0, :])
#                 coordinates = result.means[0, max_weight_index]
#             country = get_country_from_coords(coordinates[0], coordinates[1])
#             continent = get_continent_from_country(country)
#             if continent:
#                 continent_votes.append(continent)
#         except Exception as e:
#             print(e)            

#     # Trả về lục địa phổ biến nhất hoặc "Unknown" nếu không có dữ liệu
#     return max(set(continent_votes), key=continent_votes.count) if continent_votes else "Unknown"

# # Tạo UDF
# geo_bert_continent_udf = udf(geo_bert_continent, StringType())

# # Sử dụng UDF trong DataFrame
# df_with_continent = df_with_continent.withColumn(
#     "continent",
#     when(
#         col("continent").isNull(),
#         geo_bert_continent_udf(col("tweets"))
#     ).otherwise(col("continent"))
# )

In [25]:
continent_names = {code: continent['toponymName'] for code, continent in continents.items()}
country_to_continent = {country['iso']: continent_names[country['continentcode']] for country in countries.values()}

# Function to get the country code from coordinates
def get_country_from_coords(longitude, latitude, country_bboxes):
    for country_code, (country_name, (west, south, east, north)) in country_bboxes.items():
        if west <= longitude <= east and south <= latitude <= north:
            return country_code
    return "Unknown"

# Function to get the continent name from country code
def get_continent_from_country(country_code):
    return country_to_continent.get(country_code, "Unknown")

def process_user_data(json_path):
    if continent == "Unknown":
        continent_votes = []
        for tweet in tweets:
            tweet_text = tweet['tweet_content']
            tokenizer = AutoTokenizer.from_pretrained("k4tel/geo-bert-multilingual")
            tokens = tokenizer.encode(tweet_text, add_special_tokens=True)
            try:
                if len(tokens) > 450:
                    tokens = tokens[:450]
                    truncated_text = tokenizer.decode(tokens)
                                # Apply text prediction analysis for each tweet
                    result = text_prediction(model_wrapper, truncated_text, use_pipeline, True)
                else:
                    result = text_prediction(model_wrapper, tweet_text, use_pipeline, True)
            except Exception as e:
                pass

            if result and result.weights.size > 0:
                max_weight_index = np.argmax(result.weights[0, :])
                coordinates = result.means[0, max_weight_index]
                        
            continent_vote = get_continent_from_country(get_country_from_coords(coordinates[0], coordinates[1],country_bboxes))
            if continent_vote:
                continent_votes.append(continent_vote)
                    # Determine the most common continent or assign "Unknown"
        return max(set(continent_votes), key=continent_votes.count) if continent_votes else "Unknown"
    else:
        return continent
process_user_udf = udf(process_user_data, StringType())

In [26]:
df_with_updated_continent = newDf.withColumn("continent", process_user_udf(newDf["continent"], newDf["tweets"]))

In [27]:
df_with_updated_continent.printSchema()

root
 |-- projects_following: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- category: string (nullable = true)
 |    |    |-- project: string (nullable = true)
 |-- tweets: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- comment_count: long (nullable = true)
 |    |    |-- hashtags: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- mentioned_users: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- _type: string (nullable = true)
 |    |    |    |    |-- displayname: string (nullable = true)
 |    |    |    |    |-- id: long (nullable = true)
 |    |    |    |    |-- username: string (nullable = true)
 |    |    |-- place: struct (nullable = true)
 |    |    |    |-- country: string (nullable = true)
 |    |    |    |-- countryCode: string (nullable = true)

# Tweets Sentiment Analysis

In [29]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import re
# from scipy.special import softmax

# MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

# tokenizer = AutoTokenizer.from_pretrained(MODEL)
# model = AutoModelForSequenceClassification.from_pretrained(MODEL)

# def preprocess(text):
#     text = re.sub(r'user', '', text)  # Remove usernames
#     text = re.sub(r'url', '', text)  # Remove URLs
#     text = re.sub(r'rt', '', text)   # Remove RT (retweet tag)
#     return text.strip()

In [30]:
def get_sentiment_scores(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt', truncation=True, max_length=450)
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    sentiment_index = np.argmax(scores)
    sentiment_label = config.id2label[sentiment_index]
    sentiment_score = scores[sentiment_index]
    return sentiment_label, sentiment_score

In [31]:
from pyspark.sql.functions import udf, col
from pyspark.sql.types import ArrayType, MapType, StringType, FloatType

def add_sentiment_to_tweet(tweet):
    tweet_content = tweet.get("tweet_content", "")
    sentiment_label, sentiment_score = get_sentiment_scores(tweet_content)
    # Cập nhật tweet map với thông tin cảm xúc mới
    updated_tweet = tweet.copy()
    updated_tweet["sentiment_label"] = sentiment_label
    updated_tweet["sentiment_score"] = str(sentiment_score)  # Chuyển đổi thành chuỗi nếu cần
    return updated_tweet

tweet_map_type = MapType(StringType(), StringType())

add_sentiment_udf = udf(lambda tweets: [add_sentiment_to_tweet(tweet) for tweet in tweets], ArrayType(tweet_map_type))

In [32]:
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType

# Define a UDF that checks if any string in the array is longer than 10 characters
def contains_long_tweet(tweet_array):
    return any(len(tweet) > 10 for tweet in tweet_array)
contains_long_tweet_udf = udf(contains_long_tweet, BooleanType())

# Filter DataFrame using the UDF
newDf = df_with_updated_continent.filter(contains_long_tweet_udf(col('tweets.tweet_content')))

In [33]:
df_with_sentiment = newDf.withColumn("tweets", add_sentiment_udf(col("tweets")))
df_with_sentiment.printSchema()

root
 |-- projects_following: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- category: string (nullable = true)
 |    |    |-- project: string (nullable = true)
 |-- tweets: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)
 |-- user: struct (nullable = true)
 |    |-- account_type: string (nullable = true)
 |    |-- activities_hour: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- displayName: string (nullable = true)
 |    |-- favorite_count: long (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- friends_count: long (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- intro: string (nullable = true)
 |    |-- is_protected: boolean (nullable = true)
 |    |-- is_verified: boolean (nullable = true)
 |    |-- join_date: string (nullable = true)
 |    |-- listed_count: long

# Cleaning Process

In [34]:
dfData.printSchema()

root
 |-- projects_following: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- category: string (nullable = true)
 |    |    |-- project: string (nullable = true)
 |-- tweets: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- comment_count: long (nullable = true)
 |    |    |-- hashtags: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- mentioned_users: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- _type: string (nullable = true)
 |    |    |    |    |-- displayname: string (nullable = true)
 |    |    |    |    |-- id: long (nullable = true)
 |    |    |    |    |-- username: string (nullable = true)
 |    |    |-- place: struct (nullable = true)
 |    |    |    |-- country: string (nullable = true)
 |    |    |    |-- countryCode: string (nullable = true)

In [35]:
record_count = dfData.count()
print(record_count)

7973


In [36]:
#Clean continent is null
from pyspark.sql.functions import col
# Lọc để loại bỏ các bản ghi có user.continent là null hoặc trống
dfData = dfData.filter(col("user.continent").isNotNull() & (col("user.continent") != ""))

# Nếu bạn muốn xem số bản ghi sau khi lọc
print("Số bản ghi sau khi lọc:", dfData.count())

Số bản ghi sau khi lọc: 7973


In [37]:
#Clean projects_following is null
from pyspark.sql.functions import size
# Lọc để loại bỏ các bản ghi có projects_following là list rỗng
dfData = dfData.filter(size(col("projects_following")) > 0)

# Nếu bạn muốn xem số bản ghi sau khi lọc
print("Số bản ghi sau khi lọc:", dfData.count())

Số bản ghi sau khi lọc: 7973


In [38]:
from pyspark.sql.functions import size, sum as sum_

# Tính tổng số lượng tweet trên tất cả các bản ghi
total_tweet_count = dfData.agg(sum_(size(col("tweets")))).first()[0]

print("Tổng số lượng tweets:", total_tweet_count)

Tổng số lượng tweets: 273100


In [39]:
#Remove some fields not neccessaire
from pyspark.sql.functions import col, transform, struct
new_user_struct = struct([col("user." + field) for field in dfData.select("user.*").columns if field not in ["activities_hour", "account_type", "intro", "ref", "url"]])

# Define a function to recreate the struct for each tweet, excluding the unwanted fields
def recreate_tweet(tweet):
    return struct(
        *[tweet[field] for field in ["comment_count", "hashtags", "like_count", "quote_count", "retweet_count", "tweet_content", "tweet_created_at", "tweet_id", "user", "view_count","sentiment"]]
    )

# Apply this transformation to the 'tweets' array
updated_df = dfData.withColumn("tweets", transform(col("tweets"), recreate_tweet))

# Now handle the 'user' struct as you were doing
new_user_struct = struct([col("user." + field) for field in dfData.select("user.*").columns if field not in ["activities_hour", "account_type", "intro", "ref", "url"]])
updated_df = updated_df.withColumn("user", new_user_struct)

In [40]:
#Check duplicate:

# Check for duplicate rows and keep the first occurrence
df_no_duplicates = updated_df.dropDuplicates()

subset_columns = ['user']
df_no_duplicates_subset = updated_df.dropDuplicates(subset=subset_columns)

# To count the number of duplicate rows
num_duplicates = updated_df.count() - df_no_duplicates.count()
print(f"Number of duplicate rows: {num_duplicates}")

24/01/02 00:08:31 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Number of duplicate rows: 614


In [41]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Assuming 'spark' is your SparkSession and 'df' is your DataFrame
# Drop duplicates based on all three columns
df_no_duplicates = updated_df.dropDuplicates(["projects_following", "tweets", "user"])

# Show the resulting DataFrame with duplicate users removed
df_no_duplicates.show()

+--------------------+--------------------+--------------------+
|  projects_following|              tweets|                user|
+--------------------+--------------------+--------------------+
|[{PFPs, The Super...|[{1086, [NFTGivea...|{Asia, Kevin Susa...|
|[{PFPs, Chublins ...|[{0, [Bitcoin, LA...|{Africa, Victor c...|
|    [{Dexes, Demex}]|[{20, [], 282, 3,...|{Asia, Ivan 💎, 3...|
|[{PFPs, Robotos O...|[{3, [], 32, 1, 4...|{North America, W...|
|[{PFPs, CHAD DOGE...|[{0, [], 5, 0, 0,...|{Africa, Luscious...|
|[{PFPs, Alice and...|[{0, [], 0, 0, 0,...|{Asia, LUNGLEE.et...|
|[{Yield, Kalmy App}]|[{0, [], 4, 1, 0,...|{Europe, roadtoth...|
|[{Art, Zerion Gen...|[{1, [], 1, 0, 0,...|{North America, D...|
|[{Gaming, Habitat...|[{2, [], 4, 0, 1,...|{Asia, JQuantAnal...|
|[{Services, Kleros}]|[{0, [], 1, 0, 0,...|{North America, L...|
|[{Cross Chain, Ru...|[{0, [Solana, Sol...|{Africa, carl001@...|
|   [{Art, Bye BanX}]|[{0, [], 0, 0, 50...|{Europe, Dream Ko...|
|   [{Cexes, Bitkub}]|[{0,

In [42]:
df_no_duplicates.printSchema()

root
 |-- projects_following: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- category: string (nullable = true)
 |    |    |-- project: string (nullable = true)
 |-- tweets: array (nullable = true)
 |    |-- element: struct (containsNull = false)
 |    |    |-- comment_count: long (nullable = true)
 |    |    |-- hashtags: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- like_count: long (nullable = true)
 |    |    |-- quote_count: long (nullable = true)
 |    |    |-- retweet_count: long (nullable = true)
 |    |    |-- tweet_content: string (nullable = true)
 |    |    |-- tweet_created_at: string (nullable = true)
 |    |    |-- tweet_id: long (nullable = true)
 |    |    |-- user: string (nullable = true)
 |    |    |-- view_count: long (nullable = true)
 |    |    |-- sentiment: struct (nullable = true)
 |    |    |    |-- label: string (nullable = true)
 |    |    |    |-- score: double (null

In [43]:
df_no_duplicates.columns

['projects_following', 'tweets', 'user']

In [44]:
df_no_duplicates.count()

7359

In [45]:
# import pandas as pd
# pandas_df = df_no_duplicates.toPandas()

In [46]:
# pandas_df

In [47]:
#score 
from pyspark.sql.functions import explode, avg, col, struct

from pyspark.sql.functions import explode, avg, col, when
from pyspark.sql.types import IntegerType

# Function to convert sentiment labels to numeric values
def sentiment_to_numeric(label):
    return when(label == 'positive', 1).when(label == 'neutral', 0).when(label == 'negative', -1)

# Tính điểm cảm xúc trung bình cho mỗi người dùng
exploded_df = df_no_duplicates.select("user", explode("tweets").alias("tweet"))
numeric_sentiments_df = exploded_df.withColumn("numeric_sentiment", sentiment_to_numeric(col("tweet.sentiment.label")))
average_sentiment = numeric_sentiments_df.groupBy("user").agg(avg("numeric_sentiment").alias("score"))

# Ghép kết quả vào DataFrame gốc
updated_df = df_no_duplicates.join(average_sentiment, "user")

# Cập nhật cấu trúc 'user' để bao gồm trường 'score'
final_df = updated_df.withColumn("user", struct(col("user.*"), col("score")))

# Xóa cột 'score' nếu không cần thiết
final_df = final_df.drop("score")

# Hiển thị kết quả
final_df.select('user.score').show()

+--------------------+
|               score|
+--------------------+
|               0.425|
|0.034482758620689655|
| 0.40540540540540543|
| 0.34146341463414637|
|-0.13157894736842105|
| 0.18181818181818182|
| 0.09433962264150944|
| 0.22580645161290322|
|                 0.0|
|            -0.09375|
|  0.5483870967741935|
|  0.5833333333333334|
|  0.2571428571428571|
|               0.125|
|-0.03846153846153...|
| 0.08108108108108109|
|                 0.2|
|             0.15625|
|0.027777777777777776|
|              0.4375|
+--------------------+
only showing top 20 rows



In [65]:
final_df.printSchema()

root
 |-- user: struct (nullable = false)
 |    |-- continent: string (nullable = true)
 |    |-- displayName: string (nullable = true)
 |    |-- favorite_count: long (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- friends_count: long (nullable = true)
 |    |-- id: long (nullable = true)
 |    |-- is_protected: boolean (nullable = true)
 |    |-- is_verified: boolean (nullable = true)
 |    |-- join_date: string (nullable = true)
 |    |-- listed_count: long (nullable = true)
 |    |-- location: string (nullable = true)
 |    |-- media_count: long (nullable = true)
 |    |-- status_count: long (nullable = true)
 |    |-- username: string (nullable = true)
 |    |-- score: double (nullable = true)
 |-- projects_following: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- category: string (nullable = true)
 |    |    |-- project: string (nullable = true)
 |-- tweets: array (nullable = true)
 |    |-- element: struct (con

In [111]:
pandas_df = final_df.toPandas()

In [112]:
pandas_df.head()

,user,projects_following,tweets
0,"(Oceania, Mikes.realm, 12821, 2738, 1588, 1432...","[(PFPs, POTA Genesis), (Art, Elemental Faces b...","[(6, [], 31, 0, 11, ∧ ∧ — ̳͟͞͞ • ‿ • — ̳͟͞͞ lt..."
1,"(Africa, Kuzma | NFT CREATOR Tez/eth/SOL🎨🍵 | N...","[(PFPs, PharaGoddess-NFT), (Art, LA FUSION PRI...","[(1, [NFT_NYC, NFTNYC, NFT_NYC2024], 1, 0, 0, ..."
2,"(North America, BTabs, 16847, 6708, 980, 14654...","[(Memberships, EZU), (Memberships, IRL Psyched...","[(105, [], 611, 51, 254, tedxucla “ mental hea..."
3,"(Asia, uehara, 9819, 3569, 2300, 2852790708, N...","[(PFPs, Owls of Fortune)]","[(16, [UndeadArmy, RiseAndShine], 191, 2, 37, ..."
4,"(Asia, Yell 😮, 29627, 6039, 3119, 144263655323...","[(Art, fvckheads), (Art, The Lobstars), (PFPs,...","[(1, [askwhy], 18, 0, 2, early bull run vibe s..."


In [116]:
print(pandas_df.head())


                                                user  \
0  (Oceania, Mikes.realm, 12821, 2738, 1588, 1432...   
1  (Africa, Kuzma | NFT CREATOR Tez/eth/SOL🎨🍵 | N...   
2  (North America, BTabs, 16847, 6708, 980, 14654...   
3  (Asia, uehara, 9819, 3569, 2300, 2852790708, N...   
4  (Asia, Yell 😮, 29627, 6039, 3119, 144263655323...   

                                  projects_following  \
0  [(PFPs, POTA Genesis), (Art, Elemental Faces b...   
1  [(PFPs, PharaGoddess-NFT), (Art, LA FUSION PRI...   
2  [(Memberships, EZU), (Memberships, IRL Psyched...   
3                          [(PFPs, Owls of Fortune)]   
4  [(Art, fvckheads), (Art, The Lobstars), (PFPs,...   

                                              tweets  
0  [(6, [], 31, 0, 11, ∧ ∧ — ̳͟͞͞ • ‿ • — ̳͟͞͞ lt...  
1  [(1, [NFT_NYC, NFTNYC, NFT_NYC2024], 1, 0, 0, ...  
2  [(105, [], 611, 51, 254, tedxucla “ mental hea...  
3  [(16, [UndeadArmy, RiseAndShine], 191, 2, 37, ...  
4  [(1, [askwhy], 18, 0, 2, early bull run vibe s..

In [117]:
#convert to suitable structure

from pyspark.sql import Row

from pyspark.sql import Row

def row_to_dict(row):
    if isinstance(row, Row):
        return row.asDict()
    return row

def validate_user_field(row):
    user = row.get('user', {})
    if isinstance(user, Row):
        user = user.asDict()  # Chuyển đổi Row thành dict

    required_keys = ['continent', 'displayName', 'favorite_count', 'followers_count', 'friends_count', 'id', 'is_protected', 'is_verified', 'join_date', 'listed_count', 'location', 'media_count', 'status_count', 'username', 'score']
    for key in required_keys:
        user.setdefault(key, None)
    
    return user

# Apply this function to the DataFrame
pandas_df['user'] = pandas_df.apply(validate_user_field, axis=1)


def validate_projects_following_field(row):
    projects = row.get('projects_following', [])
    if not isinstance(projects, list):
        projects = []  # Chuyển đổi thành danh sách nếu không phải

    valid_projects = [row_to_dict(project) for project in projects if isinstance(project, (dict, Row))]
    return valid_projects

pandas_df['projects_following'] = pandas_df.apply(validate_projects_following_field, axis=1)

def validate_tweet_field(tweet):
    if 'sentiment' in tweet:
        tweet['sentiment'] = row_to_dict(tweet['sentiment'])
    return tweet

for index, row in pandas_df.iterrows():
    tweets = row.get('tweets', [])
    if isinstance(tweets, list):
        pandas_df.at[index, 'tweets'] = [validate_tweet_field(row_to_dict(tweet)) for tweet in tweets if isinstance(tweet, (dict, Row))]

In [118]:
pandas_df.head()

,user,projects_following,tweets
0,"{'continent': 'Oceania', 'displayName': 'Mikes...","[{'category': 'PFPs', 'project': 'POTA Genesis...","[{'comment_count': 6, 'hashtags': [], 'like_co..."
1,"{'continent': 'Africa', 'displayName': 'Kuzma ...","[{'category': 'PFPs', 'project': 'PharaGoddess...","[{'comment_count': 1, 'hashtags': ['NFT_NYC', ..."
2,"{'continent': 'North America', 'displayName': ...","[{'category': 'Memberships', 'project': 'EZU'}...","[{'comment_count': 105, 'hashtags': [], 'like_..."
3,"{'continent': 'Asia', 'displayName': 'uehara',...","[{'category': 'PFPs', 'project': 'Owls of Fort...","[{'comment_count': 16, 'hashtags': ['UndeadArm..."
4,"{'continent': 'Asia', 'displayName': 'Yell 😮',...","[{'category': 'Art', 'project': 'fvckheads'}, ...","[{'comment_count': 1, 'hashtags': ['askwhy'], ..."


# Elasticsearch

In [49]:
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch(
    ['http://34.143.255.36:9200/'],
    basic_auth=('elastic', 'elastic2023'),
    request_timeout=500
)

In [50]:
es.info()

ObjectApiResponse({'name': 'es01', 'cluster_name': 'hust-cluster', 'cluster_uuid': 'G2cWi6pkRIiHGhRX4hPOyA', 'version': {'number': '8.9.1', 'build_flavor': 'default', 'build_type': 'tar', 'build_hash': 'a813d015ef1826148d9d389bd1c0d781c6e349f0', 'build_date': '2023-08-10T05:02:32.517455352Z', 'build_snapshot': False, 'lucene_version': '9.7.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [56]:
def docs_generator(df):
    for index, document in df.iterrows():
        yield {
            "_index": "your_index_name",  # Thay thế với tên index của bạn
            "_source": document.to_dict(),
        }

# Index dữ liệu vào Elasticsearch
try:
    response = helpers.bulk(es, docs_generator(final_df))
    print(f"Bulk indexing completed. Indexed {response[0]} documents.")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: 'DataFrame' object has no attribute 'iterrows'


In [58]:
pandas_df = final_df.toPandas()

In [59]:
pandas_df

,user,projects_following,tweets
0,"(Oceania, Mikes.realm, 12821, 2738, 1588, 1432...","[(PFPs, POTA Genesis), (Art, Elemental Faces b...","[(6, [], 31, 0, 11, ∧ ∧ — ̳͟͞͞ • ‿ • — ̳͟͞͞ lt..."
1,"(Africa, Kuzma | NFT CREATOR Tez/eth/SOL🎨🍵 | N...","[(PFPs, PharaGoddess-NFT), (Art, LA FUSION PRI...","[(1, [NFT_NYC, NFTNYC, NFT_NYC2024], 1, 0, 0, ..."
2,"(North America, BTabs, 16847, 6708, 980, 14654...","[(Memberships, EZU), (Memberships, IRL Psyched...","[(105, [], 611, 51, 254, tedxucla “ mental hea..."
3,"(Asia, uehara, 9819, 3569, 2300, 2852790708, N...","[(PFPs, Owls of Fortune)]","[(16, [UndeadArmy, RiseAndShine], 191, 2, 37, ..."
4,"(Asia, Yell 😮, 29627, 6039, 3119, 144263655323...","[(Art, fvckheads), (Art, The Lobstars), (PFPs,...","[(1, [askwhy], 18, 0, 2, early bull run vibe s..."
...,...,...,...
7258,"(North America, StrayCat, 19042, 535, 3045, 91...","[(Memberships, The Immortals)]","[(3, [Moonbears, Casino], 13, 1, 4, made custo..."
7259,"(North America, luana, 86034, 16078, 978, 1055...","[(Art, Storm's editions)]","[(42, [], 11293, 26, 1848, anxiety mixed mediu..."
7260,"(Asia, ??????, 4550, 95, 190, 1029864087935766...","[(Bridge, pNetwork)]","[(0, [Luna], 0, 0, 19, rt user price luna luna..."
7261,"(North America, Nook.jpeg ⚔️🔮🐍🤖🥷, 26040, 8299,...","[(Gaming, A.N.I.M.O), (Memberships, Deadfrenz ...","[(68, [PortfolioDay], 550, 17, 127, portfoliod..."


In [61]:
from elasticsearch import Elasticsearch, helpers

# Connect to Elasticsearch
es = Elasticsearch(
    ["http://34.143.255.36:9200/"],
    basic_auth=('elastic', 'elastic2023')
)

# Function to generate Elasticsearch documents from Pandas DataFrame
def docs_generator(df):
    for index, row in df.iterrows():
        yield {
            "_index": "your_index_name",  # Replace with your index name
            "_id": index,  # Optionally set a unique ID for each document
            "_source": row.to_dict(),
        }

# Bulk index the data
try:
    response = helpers.bulk(es, docs_generator(pandas_df))
    print(f"Bulk indexing completed. {response[0]} documents indexed.")
except Exception as e:
    print(f"An error occurred: {e}")

An error occurred: 500 document(s) failed to index.


In [122]:
from elasticsearch import Elasticsearch, helpers

# Connect to Elasticsearch
es = Elasticsearch(
    ["http://34.143.255.36:9200/"],
    basic_auth=('elastic', 'elastic2023')
)

# Define index name
index_name = 'ndycuong11'

# Delete the index if it exists
es.indices.delete(index=index_name, ignore=[400, 404])

# Create a new index with the correct mappings
es.indices.create(index=index_name, body={
    'mappings': {
        'properties': {
            'user': {
                'properties': {
                    'continent': {'type': 'keyword'},
                    'displayName': {'type': 'text'},
                    'favorite_count': {'type': 'long'},
                    'followers_count': {'type': 'long'},
                    'friends_count': {'type': 'long'},
                    'id': {'type': 'keyword'},
                    'is_protected': {'type': 'boolean'},
                    'is_verified': {'type': 'boolean'},
                    'join_date': {'type': 'date', 'format': 'strict_date_optional_time||epoch_millis'},
                    'listed_count': {'type': 'long'},
                    'location': {'type': 'text'},
                    'media_count': {'type': 'long'},
                    'status_count': {'type': 'long'},
                    'username': {'type': 'text'},
                    'score': {'type': 'double'}
                }
            },
            'projects_following': {
                'type': 'nested',
                'properties': {
                    'category': {'type': 'keyword'},
                    'project': {'type': 'keyword'}
                }
            },
            'tweets': {
                'type': 'nested',
                'properties': {
                    'comment_count': {'type': 'long'},
                    'hashtags': {'type': 'keyword'}, 
                    'like_count': {'type': 'long'},
                    'quote_count': {'type': 'long'},
                    'retweet_count': {'type': 'long'},
                    'tweet_content': {'type': 'text'},
                    'tweet_created_at': {'type': 'date', 'format': 'strict_date_optional_time||epoch_millis'},
                    'tweet_id': {'type': 'keyword'},
                    'user': {'type': 'text'},
                    'view_count': {'type': 'long'},
                    'sentiment': {
                        'properties': {
                            'label': {'type': 'keyword'},
                            'score': {'type': 'double'}
                        }
                    }
                }
            }
        }
    }
}, ignore=400)

# Prepare data for bulk indexing
action_list = []
for index, row in pandas_df.iterrows():
    record = {
        '_op_type': 'index',
        '_index': index_name,
        '_source': row.to_dict()
    }
    action_list.append(record)

# Bulk index the data
try:
    success, failed = helpers.bulk(es, action_list, stats_only=False, raise_on_error=False)
    print(f"Bulk indexing completed. {success} documents indexed successfully.")
    print(f"Number of failed documents: {len(failed)}")
    for i, fail in enumerate(failed, start=1):
        print(f"Fail {i}: {fail}")
except Exception as e:
    print(f"An error occurred: {e}")

/tmp/ipykernel_4174015/1937390818.py:13: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index=index_name, ignore=[400, 404])
/tmp/ipykernel_4174015/1937390818.py:16: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name, body={


Bulk indexing completed. 7263 documents indexed successfully.
Number of failed documents: 0


In [123]:
try:
    response = es.cluster.health()
    print("Cluster Health:", response)
except Exception as e:
    print("Error connecting to Elasticsearch:", e)

Cluster Health: {'cluster_name': 'hust-cluster', 'status': 'yellow', 'timed_out': False, 'number_of_nodes': 3, 'number_of_data_nodes': 3, 'active_primary_shards': 168, 'active_shards': 237, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 99, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 70.53571428571429}


In [52]:
# import subprocess

# try:
#     # Define the curl command
#     curl_cmd = 'curl -X GET "http://34.143.255.36:9200/_cat/shards?h=index,shard,prirep,state,unassigned.reason"'

#     # Run the curl command and capture the output
#     result = subprocess.run(curl_cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

#     # Filter the output for UNASSIGNED shards
#     unassigned_shards = [line for line in result.stdout.split('\n') if 'UNASSIGNED' in line]
    
#     # Print the unassigned shards
#     for shard in unassigned_shards:
#         print(shard)
# except subprocess.CalledProcessError as e:
#     print("An error occurred:", e.stderr)

In [59]:
#(base) group02it5384@bigdata-01:~$ curl -X GET "http://34.143.255.36:9200/_cat/shards?h=index,shard,prirep,state,unassigned.reason" -u elastic:elastic2023